<a href="https://colab.research.google.com/github/JS-DS-coder/CienciaDeDatos2021/blob/master/Proyecto_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Proyecto Final**

Jairo Salazar

Carnet : 21000887 

***Parte I - Analisis exploratorio de Datos***


 

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb



In [87]:
#carga de dataset y slicin de los datos
def slicingDataset (ds,inicio,porcentaje):
  cantidad = int(np.round(ds.shape[0] * porcentaje,0))
  return ds[inicio:inicio+cantidad,0:6]
  


Dataset = np.load('proyecto_training_data.npy')
print (Dataset.shape)
dsEntrenamiento = slicingDataset(Dataset,0,0.80)
dsTest = slicingDataset(Dataset,int(dsEntrenamiento.shape[0])-1,0.20)
print (dsTest.shape)
print (dsEntrenamiento.shape)

(1460, 6)
(292, 6)
(1168, 6)


In [98]:
#Analisis Exploratorio de los datasets
#calculando media, valor maximo, valor minimo, rango, desviacion estandard.
def dimensiones (ds,columna,nombre):
  dimension = ds[0:ds.shape[0],columna-1:columna]
  media = np.mean(dimension)
  valmax = np.max(dimension)
  valmin = np.min(dimension)
  desvstd  = np.std(dimension)
  rango    = np.ptp(dimension)
  print('-------------------------------')
  print('        ', nombre)
  print ('Media: ',media)
  print ('Valor Maximo: ',valmax)
  print ('Valor Minimo: ',valmin)
  print ('Rango: ',rango)
  print ('Desviacion Standard: ',desvstd)
  sb.set_theme()
  print(dsEntrenamiento.shape)
  #sb.displot(dimension,kde=True,x=nombre)


print(dsEntrenamiento.shape)
dimensiones(dsEntrenamiento,1,"Precio de Venta")


(1168, 6)
-------------------------------
         Precio de Venta
Media:  180590.27739726027
Valor Maximo:  755000.0
Valor Minimo:  34900.0
Rango:  720100.0
Desviacion Standard:  78781.95108033362
(1168, 6)
